# Run this notebook on the Demand forecast cluster

In [0]:
%pip install lightgbm

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

In [0]:
%run /Users/ebizindavyi@delhaize.be/Evaluation/Evaluation_functions

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import lightgbm as lgb
from reusable.featurestore.features_schoolholidays import school_holidays
from pyspark.sql import functions as f


## Evaluation Params

In [0]:
actual_cols = ['Dry Actuals', 'Dry Actuals']
fc_cols = ['Dry Fc', 'LGBM_FC']
error_suffixes = ['Dry_WOW', 'Dry_LGBM']
error_to_calc = ["Bias", "AbsError", "SqError","RelAbsError"]

## Load inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)


In [0]:
# calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
# inbound_2022_dry = calendar_2022.merge(inbound_2022_df[['Dates','Dry Actuals']], on='Dates',how='left').fillna(0).copy()

### Get holidays

In [0]:
flemish_holidays = (
  school_holidays()
  .filter(
    (f.col('is_dutch_region_holiday')==1)
    &
    (f.col('holiday_name').isNull())
    )
  .withColumn('holiday', f.lit('flemish_region_holiday'))
  .withColumn('ds', f.to_date(f.col('date')))
  .select('holiday','ds')
  .toPandas()
)

walloon_holidays =(
  school_holidays()
  .filter(
    (f.col('is_french_region_holiday')==1)
    &
    (f.col('holiday_name').isNull())
    )
  .withColumn('holiday', f.lit('walloon_region_holiday'))
  .withColumn('ds', f.to_date(f.col('date')))
  .select('holiday','ds')
  .toPandas()
)

holidays = pd.concat((flemish_holidays, walloon_holidays))


## LightGBM


##Parameters

In [0]:
#lgbm parameter
backtest_start_date = pd.to_datetime('2022-01-03')
backtest_end_date =  pd.to_datetime('2022-06-30')#('2022-09-30')

#shift periods to create features(lags)
shift_period_1 = 7
shift_period_2 = 14
shift_period_3 = 21

# Initialize lgbm parameters 
params = {
    'objective': 'regression',  
    'metric': 'rmse',           
    'boosting_type': 'gbdt',    
    
}
# variables 
#pred_period = 1
# end_horizon = 7
# fc_freq=1

horizon = 8 #length of the forecasting
freq_forecast = 1
expected_pred = 1
# future_dates=1

training_period= 180

### Backtest

In [0]:
backtest_results = pd.DataFrame()

inbound_2022_train = inbound_2022_df[['Dates', 'Dry Actuals']].copy() #original filtered dataset, no sundays&NANs

inbound_2022_train['x_7days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_1)
inbound_2022_train['x_14days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_2)
inbound_2022_train['x_21days'] = inbound_2022_df['Dry Actuals'].shift(shift_period_3)

# Iterate through each backtesting period
while backtest_start_date + dt.timedelta(days=expected_pred) <= backtest_end_date:
  print(f"Backtesting on: {backtest_start_date}")
  
  start_train_date = backtest_start_date
  end_train_date = start_train_date + dt.timedelta(days=training_period)
  
  print(f"Backtesting training ending on: {end_train_date}") 

  inbound_2022_dry_lgbm_train = inbound_2022_train[(inbound_2022_train['Dates'] >= start_train_date) &(inbound_2022_train['Dates'] < end_train_date)].copy()
  print(f"print the shape:{inbound_2022_dry_lgbm_train.shape} ")
  # Convert Dates column to datetime
  inbound_2022_dry_lgbm_train['Dates'] = pd.to_datetime(inbound_2022_dry_lgbm_train['Dates'])
    
  inbound_2022_dry_lgbm_train = inbound_2022_dry_lgbm_train.dropna()
  features_values_train=inbound_2022_dry_lgbm_train.iloc[:, -3:]

  # Filter the dates with values in Dry Actuals 
  features_train = features_values_train.copy() 
  target = inbound_2022_dry_lgbm_train['Dry Actuals']
  
  # Create LightGBM dataset
  lgb_train = lgb.Dataset(features_train, label=target)
    
  # Model training
  m = lgb.train(params, lgb_train)

  # Prediction for the current period
  future_dates = pd.date_range(start=end_train_date +dt.timedelta(days=horizon), periods= expected_pred, freq='D')
                                                                 
  future_features = pd.DataFrame({'ds': future_dates})
  future_features['ds'] = pd.to_datetime(future_features['ds'])

  future_features = inbound_2022_train[inbound_2022_train['Dates'].isin(future_dates)].iloc[:, -3:].values ### Changed the source df
  prediction_dates = inbound_2022_train[inbound_2022_train['Dates'].isin(future_dates)]['Dates'] ### New variable to extract valid dates from the future date range
  
  #forecast using lgbm model m

  forecast = m.predict(future_features, num_iteration=m.best_iteration, predict_disable_shape_check=True)

  lgbm_pred = pd.DataFrame({'ds': prediction_dates, 'yhat': forecast}) ### Use dates form df rather than the original date range
    
  backtest_results = pd.concat([backtest_results, lgbm_pred])
    
  # Increase training period after each iterations 
  backtest_start_date += dt.timedelta(days=expected_pred)
print("Backtesting completed.")


Backtesting on: 2022-01-03 00:00:00
Backtesting training ending on: 2022-07-02 00:00:00
print the shape:(145, 5) 
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027007 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 129
[LightGBM] [Info] Number of data points in the train set: 124, number of used features: 3
[LightGBM] [Info] Start training from score 65260.620968
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

In [0]:
backtest_results

,ds,yhat
152,2022-07-11,48742.631725
153,2022-07-12,68594.568622
154,2022-07-13,55767.381227
155,2022-07-14,48936.878418
156,2022-07-15,72342.991970
...,...,...
293,2022-12-27,53001.629987
294,2022-12-28,53968.439296
295,2022-12-29,57989.157829
296,2022-12-30,56864.517431


### Reformat backtest

In [0]:
backtest_results['Dates']= backtest_results['ds']
backtest_results['LGBM_FC']= backtest_results['yhat']
backtest_df = backtest_results[['Dates','LGBM_FC']].reset_index(drop=True).copy()
backtest_df['Dates'] = pd.to_datetime(backtest_df['Dates'])

### Join the backtest to the inbound

In [0]:
###Remove the data quality issues where FC is exactly te same as actuals

backtest_df_merge = pd.merge(inbound_2022_df[inbound_2022_df['Dry Fc']!=inbound_2022_df['Dry Actuals']],backtest_df, on="Dates", how='inner')[['Dates','Dry Actuals','Dry Fc','LGBM_FC']]
backtest_df_merge

,Dates,Dry Actuals,Dry Fc,LGBM_FC
0,2022-07-11,72446.0,80053.600000,48742.631725
1,2022-07-12,57417.0,54939.360000,68594.568622
2,2022-07-13,46396.0,53995.500000,55767.381227
3,2022-07-14,44223.0,45738.981920,48936.878418
4,2022-07-15,41982.0,40023.074000,72342.991970
...,...,...,...,...
140,2022-12-27,63764.0,71459.270000,53001.629987
141,2022-12-28,47680.0,71916.320000,53968.439296
142,2022-12-29,39552.0,60346.346000,57989.157829
143,2022-12-30,61119.0,52203.888000,56864.517431


## Evaluate

### Create error columns

In [0]:
for error in  error_to_calc:
  for i in range(len(actual_cols)):
    calculate_errors(df_eval= backtest_df_merge, actuals_col=actual_cols[i],
                      fc_col= fc_cols[i],error_to_calculate=error,error_colname_suffix=error_suffixes[i])
    

In [0]:
backtest_df_merge


,Dates,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,Outside_range_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_LGBM
0,2022-07-11,72446.0,80053.600000,48742.631725,7607.600000,-23703.368275,7607.600000,23703.368275,5.787558e+07,5.618497e+08,0.105011,True,0.327187,True
1,2022-07-12,57417.0,54939.360000,68594.568622,-2477.640000,11177.568622,2477.640000,11177.568622,6.138700e+06,1.249380e+08,0.043152,False,0.194674,True
2,2022-07-13,46396.0,53995.500000,55767.381227,7599.500000,9371.381227,7599.500000,9371.381227,5.775240e+07,8.782279e+07,0.163796,True,0.201987,True
3,2022-07-14,44223.0,45738.981920,48936.878418,1515.981920,4713.878418,1515.981920,4713.878418,2.298201e+06,2.222065e+07,0.034280,False,0.106593,True
4,2022-07-15,41982.0,40023.074000,72342.991970,-1958.926000,30360.991970,1958.926000,30360.991970,3.837391e+06,9.217898e+08,0.046661,False,0.723191,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2022-12-27,63764.0,71459.270000,53001.629987,7695.270000,-10762.370013,7695.270000,10762.370013,5.921718e+07,1.158286e+08,0.120684,True,0.168784,True
141,2022-12-28,47680.0,71916.320000,53968.439296,24236.320000,6288.439296,24236.320000,6288.439296,5.873992e+08,3.954447e+07,0.508312,True,0.131888,True
142,2022-12-29,39552.0,60346.346000,57989.157829,20794.346000,18437.157829,20794.346000,18437.157829,4.324048e+08,3.399288e+08,0.525747,True,0.466150,True
143,2022-12-30,61119.0,52203.888000,56864.517431,-8915.112000,-4254.482569,8915.112000,4254.482569,7.947922e+07,1.810062e+07,0.145865,True,0.069610,False


In [0]:

days_out_range= backtest_df_merge.groupby('Dates').agg({'Outside_range_Dry_LGBM':'sum'}).reset_index()
days_out_range

,Dates,Outside_range_Dry_LGBM
0,2022-07-11,1
1,2022-07-12,1
2,2022-07-13,1
3,2022-07-14,1
4,2022-07-15,1
...,...,...
140,2022-12-27,1
141,2022-12-28,1
142,2022-12-29,1
143,2022-12-30,0


In [0]:
filter_df = days_out_range[days_out_range['Outside_range_Dry_LGBM'] != 0]
filter_df

,Dates,Outside_range_Dry_LGBM
0,2022-07-11,1
1,2022-07-12,1
2,2022-07-13,1
3,2022-07-14,1
4,2022-07-15,1
...,...,...
138,2022-12-24,1
139,2022-12-26,1
140,2022-12-27,1
141,2022-12-28,1


### Calculate global metrics

In [0]:
global_metrics_dry = describe_inbound_fc(inbound_fc=backtest_df_merge, cols_to_keep=['Dry Actuals','Dry Fc','LGBM_FC','Bias_Dry_WOW','Bias_Dry_LGBM', 'AbsError_Dry_WOW','AbsError_Dry_LGBM','SqError_Dry_WOW','SqError_Dry_LGBM','RelAbsError_Dry_WOW','RelAbsError_Dry_LGBM', 'Outside_range_Dry_WOW', 'Outside_range_Dry_LGBM'])

<command-203330119889549>:25: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  described_df = inbound_fc.describe(include='all')[cols_to_keep].rename_axis('Metrics').reset_index(drop=False).copy()


In [0]:
global_metrics_dry

,Metrics,Dry Actuals,Dry Fc,LGBM_FC,Bias_Dry_WOW,Bias_Dry_LGBM,AbsError_Dry_WOW,AbsError_Dry_LGBM,SqError_Dry_WOW,SqError_Dry_LGBM,RelAbsError_Dry_WOW,RelAbsError_Dry_LGBM,Outside_range_Dry_WOW,Outside_range_Dry_LGBM
0,count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,1.450000e+02,1.450000e+02,145.000000,145.000000,145,145
1,unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
3,freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92,117
4,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mean,65536.255172,67735.871033,62463.570844,2199.615861,-3072.684328,6445.669793,17753.744283,1.044196e+08,6.680820e+08,0.125101,0.284208,NaN,NaN
7,std,25203.417497,22967.146739,10461.330437,10013.629792,25752.951018,7956.727553,18850.388512,3.828211e+08,1.781908e+09,0.243816,0.362927,NaN,NaN
8,min,20444.000000,35712.572000,39917.571760,-59204.840000,-119391.227436,4.000000,23.247397,1.600000e+01,5.404415e+02,0.000081,0.000490,NaN,NaN
9,25%,50038.000000,53995.500000,53688.466398,-2477.640000,-12116.786618,2163.796000,7033.595548,4.682013e+06,4.947147e+07,0.033877,0.120646,NaN,NaN


### Visualize profiles

In [0]:
plot_two_inbound_fc(inbound_df=backtest_df_merge, actuals_col='Dry Actuals', fc_col_1='Dry Fc', fc_col_2= 'LGBM_FC', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

### Visualize errors

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='Bias_Dry_WOW', error_col_2= 'Bias_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')

In [0]:
plot_inbound_two_errors(inbound_df=backtest_df_merge, error_col_1='AbsError_Dry_WOW', error_col_2= 'AbsError_Dry_LGBM', show_out_of_range=True, date_col='Dates', outside_range_col_1='Outside_range_Dry_WOW', outside_range_col_2='Outside_range_Dry_LGBM')